<a href="https://colab.research.google.com/github/choons1k2/2022-Fall-Opensource-AI/blob/main/xgboost_%EC%B5%9C%EB%AF%BC%EC%84%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
f_path = '/content/heart.csv'

import pandas as pd
heart = pd.read_csv(f_path)


#OneHotEncoding
heart = pd.get_dummies(heart)
heart.count()

Mounted at /content/drive


Age                  918
RestingBP            918
Cholesterol          918
FastingBS            918
MaxHR                918
Oldpeak              918
HeartDisease         918
Sex_F                918
Sex_M                918
ChestPainType_ASY    918
ChestPainType_ATA    918
ChestPainType_NAP    918
ChestPainType_TA     918
RestingECG_LVH       918
RestingECG_Normal    918
RestingECG_ST        918
ExerciseAngina_N     918
ExerciseAngina_Y     918
ST_Slope_Down        918
ST_Slope_Flat        918
ST_Slope_Up          918
dtype: int64

In [33]:
X = heart.drop('HeartDisease', axis=1)
Y = (heart["HeartDisease"])



from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.13, random_state = 42)

In [29]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
!pip install optuna
import optuna

def RF_objective(trial: optuna.trial.Trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric':'auc',
        'booster':'gbtree',
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000, step=10),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        
        # 'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시
        "tree_method": 'exact', 'gpu_id': -1,  # CPU 사용시
        
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1),
        # 'scale_pos_weight': scale_pos_weight
        # 'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 5, 20, step=5)
        
        # "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        # 'num_parallel_tree': trial.suggest_int("num_parallel_tree", 1, 500) 추가하면 느려짐.
    }

    model = xgb.XGBClassifier(**params, random_state = 42, use_label_encoder = False, n_jobs=-1)
    
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
    f1_mean = score.mean()

    return f1_mean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
RF_study = optuna.create_study()
RF_study.optimize(RF_objective, n_trials=100)

[I 2022-11-27 12:34:05,089] A new study created in memory with name: no-name-f5dbc66e-1304-4ec5-b7d8-a8a11f8efdc4
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
/usr/local/lib/python3.7/dist-p